In [10]:
from shapely import geometry, wkt
import rasterio
import numpy as np
import rasterio.features
import geopandas as gpd
from shapely.geometry import Polygon, box
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from geopandas import GeoSeries, GeoDataFrame

shp = 'segura/Crop_Classification_Delivery.shp' 
rasterfile = 'segura/WorldCover.tif' 
path = 'segura/'



# 1) Add column 'Num_class' to be burnt in raster

In [2]:
parcels = gpd.read_file(shp)


In [3]:
parcels.C_AH2021.unique()

array(['FR', 'CIT', 'CITj', 'H', 'FRj', 'INV', 'VP', 'ALM', 'V', 'OV',
       'LBC', 'HU'], dtype=object)

In [4]:
parcels['Class_num'] = parcels['C_AH2021'].replace({ 'H':'501',
                                                     'HU':'502',
                                                     'CIT':'503',
                                                     'CITj':'504',
                                                     'FR':'505',
                                                     'FRj':'506',
                                                     'ALM':'507',
                                                     'OV':'508',
                                                     'V':'509',
                                                     'VP':'510',
                                                     'INV':'511',
                                                     'LBC':'512' })



In [5]:
parcels.head(3)

,fid,C21_M01,C21_M02,C21_M03,C21_M04,C21_M05,C21_M06,C21_M07,C21_M08,C21_M09,C20_M10,C20_M11,C20_M12,C_AH2021,Probabilit,Superficie,geometry,Class_num
0,1.0,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,FR,0.45,89244.0,POLYGON ((-1.173430929550738 38.11558639322397...,505
1,2.0,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,CIT,0.31,5111.0,POLYGON ((-1.070037647175303 38.07409478302146...,503
2,3.0,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,LAC,FR,0.46,51250.0,POLYGON ((-1.174582683891562 38.11685685592683...,505


In [6]:
#check if all classes are burnt to numbers
parcels.Class_num.unique()

array(['505', '503', '504', '501', '506', '511', '510', '507', '509',
       '508', '512', '502'], dtype=object)

# Set coord system and dtype for a raster file 

In [7]:
gdf = gpd.GeoDataFrame(parcels, crs="EPSG:4326", geometry='geometry')

gdf['Class_num'] = gdf['Class_num'].astype('int16')

gdf["geometry"] = [MultiPolygon([feature]) if isinstance(feature, Polygon) \
    else feature for feature in gdf["geometry"]]


# Save a file to geojson

In [8]:

gdf.to_file('segura/Crop_Classification_Delivery_class.geojson' , driver="GeoJSON") 


/home/jovyan/.conda/envs/py3/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
CPLE_NotSupported in driver GeoJSON does not support creation option ENCODING
CPLE_NotSupported in dataset /home/jovyan/exchange/7_EvapoTranspiration/segura/Crop_Classification_Delivery_class.geojson does not support layer creation option ENCODING
CPLE_NotSupported in dataset /home/jovyan/exchange/7_EvapoTranspiration/segura/Crop_Classification_Delivery_class.geojson does not support layer creation option ENCODING


# Rasterize vector using the shape and transform of the raster

In [11]:
raster = rasterio.open(rasterfile)

geom_value = ((geom,value) for geom, value in zip(gdf.geometry, gdf['Class_num']))

rasterized = rasterio.features.rasterize(geom_value,
                                out_shape = raster.shape,
                                transform = raster.transform,
                                all_touched = True,
                                fill = -9999,   # background value
                                dtype = np.int16)

with rasterio.open("segura/rasterized_crop.tif", "w",
        driver = "GTiff",
        transform = raster.transform,
        dtype = rasterio.int16,
        count = 1,
        crs = raster.crs,
        nodata = -9999,
        compress = 'lzw',
        width = raster.width,
        height = raster.height) as dst:
    dst.write(rasterized, indexes = 1)
    

# Compress final raster

In [ ]:

dat = rasterio.open("segura/WorldCover.tif")
profile = dat.profile.copy()
profile.update(compress='lzw')

with rasterio.open("segura/WorldCover_compressed.tif", 'w', **profile) as dst:
    for ji, window in dat.block_windows(1):
        dst.write(dat.read(window=window), window=window)